In [1]:
#!/usr/bin/env python3
"""
webscanpro.py
Merged DVWA crawler + SQLi + XSS + Auth + AccessControl tester + HTML reporting (Jinja2).
Generates: reports/webscan_report_<slugified-target>.html
CSV export removed; report ensures 'parameter' and 'evidence' are filled when possible.
"""

import os
import re
import time
import requests
from urllib.parse import urljoin, urldefrag, urlparse, parse_qs, urlencode, urlunparse
from datetime import datetime
from collections import deque
from bs4 import BeautifulSoup
from tqdm import tqdm
from jinja2 import Environment, FileSystemLoader, select_autoescape

# ==============================
# Templates / Reporting
# ==============================
REPORT_TEMPLATE = """
<!doctype html>
<html>
<head>
  <meta charset="utf-8" />
  <title>WebScanPro Report - {{ target }}</title>
  <style>
    body { font-family: Arial, sans-serif; padding: 20px; }
    table { border-collapse: collapse; width: 100%; }
    th, td { border: 1px solid #ddd; padding: 8px; }
    th { background: #f4f4f4; }
    .High { color: #a94442; font-weight: bold; }
    .Medium { color: #8a6d3b; }
    .Low { color: #3c763d; }
  </style>
</head>
<body>
  <h1>WebScanPro Report</h1>
  <p><strong>Target:</strong> {{ target }}</p>
  <p><strong>Date:</strong> {{ date }}</p>

  <h2>Findings ({{ findings|length }})</h2>
  {% if findings %}
  <table>
    <thead>
      <tr>
        <th>Type</th>
        <th>Endpoint / URL</th>
        <th>Parameter / Field</th>
        <th>Payload / Value</th>
        <th>Evidence / Error / Issues</th>
        <th>Severity</th>
      </tr>
    </thead>
    <tbody>
      {% for f in findings %}
      <tr>
        <td>{{ f.type }}</td>
        <td>{{ f.endpoint }}</td>
        <td>{{ f.param }}</td>
        <td>{{ f.payload }}</td>
        <td>{{ f.evidence }}</td>
        <td class="{{ f.severity }}">{{ f.severity }}</td>
      </tr>
      {% endfor %}
    </tbody>
  </table>
  {% else %}
    <p>No findings.</p>
  {% endif %}
</body>
</html>
"""

def severity_from_type(t):
    t = str(t or "")
    if 'SQLi' in t or 'XSS' in t:
        return 'High'
    if 'IDOR' in t or 'Session' in t or 'credentials' in t.lower() or 'Weak-Credentials' in t:
        return 'Medium'
    return 'Low'


class Reporter:
    def __init__(self, target, findings, out_dir='reports'):
        self.target = target
        self.findings = findings or []
        self.out_dir = out_dir
        os.makedirs(out_dir, exist_ok=True)

    def _slugify(self, s):
        s = re.sub(r'^\w+://', '', s)  # remove scheme
        s = s.strip().lower()
        s = re.sub(r'[^a-z0-9]+', '_', s)
        s = s.strip('_')
        return s or 'target'

    def normalize_findings(self):
        """Ensure template-friendly keys, map aliases and compute severity.
        Prioritize populating param and evidence from available fields.
        """
        out = []
        for f in self.findings:
            nf = dict(f)  # shallow copy

            # alias mapping
            if 'parameter' in nf and 'param' not in nf:
                nf['param'] = nf.get('parameter')
            if 'field' in nf and 'param' not in nf:
                nf['param'] = nf.get('field')
            if 'url' in nf and 'endpoint' not in nf:
                nf['endpoint'] = nf.get('url')
            if 'status' in nf and 'endpoint' not in nf:
                nf['endpoint'] = str(nf.get('status'))

            # defaults and payload/evidence selection
            nf['type'] = nf.get('type', 'N/A')
            nf['endpoint'] = nf.get('endpoint', nf.get('url', '-')) or '-'

            # Parameter: try many fields to fill it
            param_val = nf.get('param') or nf.get('parameter') or nf.get('field') or nf.get('value') or nf.get('username') or nf.get('param')
            nf['param'] = param_val if param_val not in (None, '') else '-'

            # Payload: prefer explicit payload or tested_value or username/password/value
            nf['payload'] = nf.get('payload') or nf.get('tested_value') or nf.get('username') or nf.get('password') or nf.get('value') or '-'

            # Evidence: prefer explicit evidence > error > issues > payload
            evidence = nf.get('evidence')
            if not evidence:
                if 'error' in nf and nf.get('error'):
                    evidence = nf.get('error')
                elif 'issues' in nf and nf.get('issues'):
                    issues = nf.get('issues')
                    evidence = ', '.join(issues) if isinstance(issues, (list, tuple)) else str(issues)
                elif nf.get('payload') and nf.get('payload') != '-':
                    evidence = f"Reflected/payload: {nf.get('payload')}"
                else:
                    # small informative fallback
                    evidence = nf.get('note') or '-'
            nf['evidence'] = evidence or '-'

            nf['severity'] = severity_from_type(nf['type'])
            out.append(nf)
        return out

    def render_html(self):
        findings = self.normalize_findings()
        env = Environment(loader=FileSystemLoader('.'), autoescape=select_autoescape(['html','xml']))
        template = env.from_string(REPORT_TEMPLATE)
        content = template.render(
            target=self.target,
            date=datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC"),
            findings=findings
        )
        slug = self._slugify(self.target)
        out_path = os.path.join(self.out_dir, f"webscan_report_{slug}.html")
        with open(out_path, 'w', encoding='utf-8') as fh:
            fh.write(content)
        return out_path

# ==============================
# Payloads & helpers
# ==============================
SQL_PAYLOADS = [
    "' OR '1'='1",
    '" OR "1"="1',
    "1'--",
    "1 OR 1=1",
    "' UNION SELECT NULL--"
]

SQL_ERROR_PATTERNS = [
    "you have an error in your sql syntax;",
    "warning: mysql",
    "unclosed quotation mark after the character string",
    "quoted string not properly terminated",
    "pg_query",
    "mysql_fetch",
    "sql error"
]

def find_sql_errors(html):
    for pattern in SQL_ERROR_PATTERNS:
        if pattern.lower() in (html or "").lower():
            return True, pattern
    return False, None

XSS_PAYLOADS = [
    "<script>alert(1)</script>",
    "<img src=x onerror=alert(1)>",
    "<svg onload=alert(1)>"
]
REFLECT_PATTERNS = [re.compile(re.escape(p), re.IGNORECASE) for p in XSS_PAYLOADS]

# ==============================
# DVWA Web Crawler
# ==============================
class DVWACrawler:
    def __init__(self, base_url, session=None, username="admin", password="password",
                 max_pages=20, delay=2):
        self.base_url = base_url.rstrip("/")
        self.max_pages = max_pages
        self.delay = delay
        self.session = session or requests.Session()
        self.visited = set()
        self.queue = deque([self.base_url])
        self.pages = {}
        self.forms = {}

        # login (best-effort)
        try:
            self.login(username, password)
        except Exception as e:
            print(f"[!] Login failed or skipped: {e}")

    def normalize_url(self, url):
        clean_url = url or ""
        clean_url = urldefrag(clean_url)[0].rstrip("/")
        return clean_url

    def login(self, username, password):
        login_url = f"{self.base_url}/login.php"
        resp = self.session.get(login_url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        token = soup.find("input", {"name": "user_token"})
        token_val = token["value"] if token else ""

        payload = {
            "username": username,
            "password": password,
            "Login": "Login",
            "user_token": token_val
        }

        login_resp = self.session.post(login_url, data=payload, timeout=10)
        if "Login failed" in login_resp.text:
            raise Exception("Login failed: check credentials or CSRF token.")
        print("✅ Login successful (if credentials valid)")

    def extract_links(self, html, page_url):
        soup = BeautifulSoup(html or "", "html.parser")
        links = []
        for a in soup.find_all("a", href=True):
            abs_url = urljoin(page_url, a["href"])
            clean_url = self.normalize_url(abs_url)
            if clean_url.startswith(self.base_url):
                links.append(clean_url)
        return links

    def extract_forms(self, html, page_url):
        soup = BeautifulSoup(html or "", "html.parser")
        forms = []
        for form in soup.find_all("form"):
            details = {
                "method": form.get("method", "get").lower(),
                "action": urljoin(page_url, form.get("action", "")),
                "inputs": []
            }
            for inp in form.find_all(["input", "textarea", "select"]):
                details["inputs"].append({
                    "name": inp.get("name"),
                    "type": inp.get("type", inp.name),
                    "value": inp.get("value", "")
                })
            forms.append(details)
        return forms

    def crawl(self):
        with tqdm(total=self.max_pages, desc="Crawling") as pbar:
            while self.queue and len(self.visited) < self.max_pages:
                url = self.normalize_url(self.queue.popleft())

                if url in self.visited:
                    continue

                try:
                    resp = self.session.get(url, timeout=10)
                    resp.raise_for_status()
                    html = resp.text
                except Exception as e:
                    print(f" Error fetching {url}: {e}")
                    self.visited.add(url)
                    continue

                self.pages[url] = html
                page_forms = self.extract_forms(html, url)
                if page_forms:
                    self.forms[url] = page_forms

                for link in self.extract_links(html, url):
                    if link not in self.visited and link not in self.queue:
                        self.queue.append(link)

                self.visited.add(url)
                print(f"✅ Crawled: {url}")
                pbar.update(1)
                time.sleep(self.delay)

        return {"pages": self.pages, "forms": self.forms, "session": self.session}

# ==============================
# SQL Injection Scanner
# ==============================
class SQLiScanner:
    def __init__(self, session, timeout=10):
        self.session = session
        self.timeout = timeout
        self.findings = []

    def test_url_params(self, url):
        parsed = urlparse(url)
        params = parse_qs(parsed.query)
        if not params:
            return
        for param in params:
            for payload in SQL_PAYLOADS:
                test_params = params.copy()
                test_params[param] = payload
                new_query = urlencode(test_params, doseq=True)
                new_url = urlunparse(parsed._replace(query=new_query))
                try:
                    resp = self.session.get(new_url, timeout=self.timeout)
                    found, pattern = find_sql_errors(resp.text)
                    if found:
                        self.findings.append({
                            "type": "SQLi-URL",
                            "url": new_url,
                            "parameter": param,
                            "payload": payload,
                            "error": pattern,
                            "evidence": f"SQL error matched: {pattern}"
                        })
                        print(f"🔥 SQLi detected in {new_url} param={param} payload={payload}")
                except Exception:
                    pass
                time.sleep(0.2)

    def test_forms(self, forms_by_url):
        for page_url, forms in forms_by_url.items():
            for form in forms:
                action = form.get("action") or page_url
                method = form.get("method", "get").lower()
                base_data = {inp["name"]: (inp.get("value") or "test")
                             for inp in form["inputs"] if inp.get("name")}
                for field in base_data:
                    for payload in SQL_PAYLOADS:
                        test_data = base_data.copy()
                        test_data[field] = payload
                        try:
                            if method == "post":
                                resp = self.session.post(action, data=test_data,
                                                         timeout=self.timeout)
                            else:
                                resp = self.session.get(action, params=test_data,
                                                        timeout=self.timeout)
                            found, pattern = find_sql_errors(resp.text)
                            if found:
                                self.findings.append({
                                    "type": "SQLi-FORM",
                                    "url": action,
                                    "field": field,
                                    "parameter": field,
                                    "payload": payload,
                                    "error": pattern,
                                    "evidence": f"SQL error matched: {pattern}"
                                })
                                print(f"🔥 SQLi detected in form {action} field={field} payload={payload}")
                        except Exception:
                            pass
                        time.sleep(0.2)

    def run(self, pages, forms):
        for url in pages.keys():
            self.test_url_params(url)
        self.test_forms(forms)
        return self.findings

# ==============================
# XSS Scanner
# ==============================
class XSSScanner:
    def __init__(self, session, timeout=10):
        self.session = session
        self.timeout = timeout
        self.findings = []

    def test_url_params(self, url):
        parsed = urlparse(url)
        params = parse_qs(parsed.query)
        if not params:
            return
        for param in params:
            for payload in XSS_PAYLOADS:
                test_params = params.copy()
                test_params[param] = payload
                new_query = urlencode(test_params, doseq=True)
                new_url = urlunparse(parsed._replace(query=new_query))
                try:
                    resp = self.session.get(new_url, timeout=self.timeout)
                    for pattern in REFLECT_PATTERNS:
                        if pattern.search(resp.text):
                            self.findings.append({
                                "type": "XSS-URL",
                                "url": new_url,
                                "parameter": param,
                                "payload": payload,
                                "evidence": "Payload reflected in response"
                            })
                            print(f"⚡ XSS detected in {new_url} param={param} payload={payload}")
                            break
                except Exception:
                    pass
                time.sleep(0.2)

    def test_forms(self, forms_by_url):
        for page_url, forms in forms_by_url.items():
            for form in forms:
                action = form.get("action") or page_url
                method = form.get("method", "get").lower()
                base_data = {inp["name"]: (inp.get("value") or "test")
                             for inp in form["inputs"] if inp.get("name")}
                for field in base_data:
                    for payload in XSS_PAYLOADS:
                        test_data = base_data.copy()
                        test_data[field] = payload
                        try:
                            if method == "post":
                                resp = self.session.post(action, data=test_data,
                                                         timeout=self.timeout)
                            else:
                                resp = self.session.get(action, params=test_data,
                                                        timeout=self.timeout)
                            for pattern in REFLECT_PATTERNS:
                                if pattern.search(resp.text):
                                    self.findings.append({
                                        "type": "XSS-FORM",
                                        "url": action,
                                        "field": field,
                                        "parameter": field,
                                        "payload": payload,
                                        "evidence": "Payload reflected in response"
                                    })
                                    print(f"⚡ XSS detected in form {action} field={field} payload={payload}")
                                    break
                        except Exception:
                            pass
                        time.sleep(0.2)

    def run(self, pages, forms):
        for url in pages.keys():
            self.test_url_params(url)
        self.test_forms(forms)
        return self.findings

# ==============================
# Authentication & Access Control Testers
# ==============================
class AuthSessionTester:
    DEFAULT_CREDENTIALS = [
        ("admin", "admin"),
        ("admin", "password"),
        ("root", "root"),
        ("test", "test")
    ]

    def __init__(self, base_url, session=None, timeout=10):
        self.base_url = base_url.rstrip("/")
        self.session = session or requests.Session()
        self.timeout = timeout
        self.findings = []

    def test_weak_credentials(self, login_path="/login.php"):
        login_url = urljoin(self.base_url, login_path.lstrip("/"))
        print(f"\n🔎 Testing Weak Credentials at {login_url}")
        for user, pwd in self.DEFAULT_CREDENTIALS:
            try:
                r = self.session.get(login_url, timeout=self.timeout)
                soup = BeautifulSoup(r.text, "html.parser")
                token = soup.find("input", {"name": "user_token"})
                token_val = token["value"] if token else ""
                data = {
                    "username": user,
                    "password": pwd,
                    "Login": "Login",
                    "user_token": token_val
                }
                resp = self.session.post(login_url, data=data, timeout=self.timeout)
                if "Logout" in resp.text or "Welcome" in resp.text:
                    evidence = f"Login succeeded for {user}:{pwd}"
                    print(f"🔥 Weak credentials found → {user}:{pwd}")
                    self.findings.append({
                        "type": "Weak-Credentials",
                        "username": user,
                        "password": pwd,
                        "parameter": user,
                        "payload": pwd,
                        "evidence": evidence
                    })
                else:
                    print(f"[-] {user}:{pwd} rejected")
            except Exception as e:
                print(f"[Error] {e}")
            time.sleep(0.5)
        return self.findings

    def check_cookie_flags(self, login_path="/login.php"):
        login_url = urljoin(self.base_url, login_path.lstrip("/"))
        print("\n🔎 Checking Cookie Flags")
        try:
            resp = self.session.get(login_url, timeout=self.timeout)
            set_cookie = resp.headers.get("set-cookie", "")
            issues = []
            if "httponly" not in set_cookie.lower():
                issues.append("Missing HttpOnly")
            if "secure" not in set_cookie.lower():
                issues.append("Missing Secure")
            if "samesite" not in set_cookie.lower():
                issues.append("Missing SameSite")
            if issues:
                evidence = ", ".join(issues)
                print("⚠️ Cookie flag issues:", evidence)
                self.findings.append({
                    "type": "Cookie-Issues",
                    "issues": issues,
                    "evidence": evidence,
                    "parameter": "-"
                })
            else:
                print("✅ Cookie flags appear secure")
        except Exception as e:
            print(f"[Error] {e}")
        return self.findings

class AccessControlTester:
    def __init__(self, base_url, session=None, timeout=10):
        self.base_url = base_url.rstrip("/")
        self.session = session or requests.Session()
        self.timeout = timeout
        self.findings = []

    def test_idor(self, path="/vulnerabilities/idor/", param="id", start=1, stop=3):
        url = urljoin(self.base_url, path.lstrip("/"))
        print(f"\n🔎 Testing IDOR at {url} (param={param})")
        for uid in range(start, stop + 1):
            try:
                resp = self.session.get(url, params={param: uid}, timeout=self.timeout)
                body = (resp.text or "").lower()
                if "unauthorized" not in body and "forbidden" not in body:
                    if "username" in body or "email" in body or "account" in body:
                        evidence = f"Accessed user data for {param}={uid}"
                        print(f"🔥 Possible IDOR → accessed data for {param}={uid}")
                        self.findings.append({
                            "type": "IDOR",
                            "url": url,
                            "param": param,
                            "parameter": param,
                            "value": uid,
                            "evidence": evidence
                        })
                    else:
                        print(f"[?] {param}={uid} returned content (needs review)")
                else:
                    print(f"[-] {param}={uid} blocked")
            except Exception as e:
                print(f"[Error] {e}")
        return self.findings

    def test_role_bypass(self, protected_paths=None):
        if protected_paths is None:
            protected_paths = ["/admin/", "/config.php", "/vulnerabilities/"]
        print("\n🔎 Testing Access Control (Role Bypass)")
        for path in protected_paths:
            url = urljoin(self.base_url, path.lstrip("/"))
            try:
                resp = requests.get(url, timeout=self.timeout)  # new session (unauth)
                if resp.status_code == 200 and "login" not in resp.text.lower():
                    evidence = f"Accessible without auth (HTTP {resp.status_code})"
                    print(f"🔥 Access Control issue → {url} accessible without login")
                    self.findings.append({
                        "type": "Access-Control",
                        "url": url,
                        "status": resp.status_code,
                        "parameter": "-",
                        "evidence": evidence
                    })
                else:
                    print(f"[-] {url} seems protected")
            except Exception as e:
                print(f"[Error] {e}")
        return self.findings

# ==============================
# Main Runner
# ==============================
if __name__ == "__main__":
    base_url = "http://localhost:8080"  # Change to your DVWA or test target

    shared_session = requests.Session()

    # Step 1: Crawl
    crawler = DVWACrawler(base_url, session=shared_session, username="admin", password="password",
                          max_pages=15, delay=1)
    crawl_results = crawler.crawl()

    # Step 2: SQL Injection Scan
    sqli_scanner = SQLiScanner(crawl_results["session"])
    sqli_findings = sqli_scanner.run(crawl_results["pages"], crawl_results["forms"])

    # Step 3: XSS Scan
    xss_scanner = XSSScanner(crawl_results["session"])
    xss_findings = xss_scanner.run(crawl_results["pages"], crawl_results["forms"])

    # Step 4: Authentication & Access Control
    auth_tester = AuthSessionTester(base_url, session=crawl_results["session"])
    auth_findings = auth_tester.test_weak_credentials()
    cookie_findings = auth_tester.check_cookie_flags()

    access_tester = AccessControlTester(base_url, session=crawl_results["session"])
    idor_findings = access_tester.test_idor(path="/vulnerabilities/idor/", param="id", start=1, stop=5)
    role_bypass_findings = access_tester.test_role_bypass()

    # Aggregate findings
    all_findings = sqli_findings + xss_findings + auth_tester.findings + access_tester.findings

    # Pretty print
    print("\n=== Vulnerability Findings ===")
    if all_findings:
        for f in all_findings:
            print(f"[{f.get('type')}]")
            if 'url' in f:
                print(f"   URL: {f.get('url')}")
            if 'parameter' in f:
                print(f"   Parameter: {f.get('parameter')}")
            if 'param' in f:
                print(f"   Param (alias): {f.get('param')}")
            if 'field' in f:
                print(f"   Field: {f.get('field')}")
            if 'username' in f:
                print(f"   Username: {f.get('username')}")
            if 'password' in f:
                print(f"   Password: {f.get('password')}")
            if 'value' in f:
                print(f"   Value: {f.get('value')}")
            if 'error' in f:
                print(f"   Error: {f.get('error')}")
            if 'issues' in f:
                print(f"   Issues: {f.get('issues')}")
            if 'evidence' in f:
                print(f"   Evidence: {f.get('evidence')}")
            print("-" * 50)
    else:
        print("No SQLi, XSS, authentication, or access control vulnerabilities detected.")

    # HTML Report
    try:
        reporter = Reporter(base_url, all_findings, out_dir='reports')
        report_path = reporter.render_html()
        print(f"\n📄 HTML report generated: {report_path}")
    except Exception as e:
        print(f"[Error] Failed to generate HTML report: {e}")

    print("\n=== Mitigation Tips ===")
    print("SQLi Fixes: Use parameterized queries (PreparedStatements), ORM, input validation.")
    print("XSS Fixes: Validate input, encode output, apply Content Security Policy (CSP).")
    print("Auth/Access Control: Enforce strong passwords, rate limit login attempts, set cookie flags (HttpOnly, Secure, SameSite), and apply proper authorization checks.")


Login successful (if credentials valid)


Crawling:   0%|                                                                                 | 0/15 [00:00<?, ?it/s]

✅ Crawled: http://localhost:8080


Crawling:   7%|████▊                                                                    | 1/15 [00:01<00:14,  1.03s/it]



🔎 Testing Weak Credentials at http://localhost:8080/login.php
[-] admin:admin rejected
[-] admin:password rejected
[-] root:root rejected
[-] test:test rejected

🔎 Checking Cookie Flags
⚠️ Cookie flag issues: Missing HttpOnly, Missing Secure, Missing SameSite

🔎 Testing IDOR at http://localhost:8080/vulnerabilities/idor/ (param=id)
[?] id=1 returned content (needs review)
[?] id=2 returned content (needs review)
[?] id=3 returned content (needs review)
[?] id=4 returned content (needs review)
[?] id=5 returned content (needs review)

🔎 Testing Access Control (Role Bypass)
[-] http://localhost:8080/admin/ seems protected
[-] http://localhost:8080/config.php seems protected
🔥 Access Control issue → http://localhost:8080/vulnerabilities/ accessible without login

=== Vulnerability Findings ===
[Cookie-Issues]
   Parameter: -
   Issues: ['Missing HttpOnly', 'Missing Secure', 'Missing SameSite']
   Evidence: Missing HttpOnly, Missing Secure, Missing SameSite
-------------------------------